# Korrelationen

In den folgenden Notebook geht es darum einen Datensatz zu laden und einfache Inferenzstatistik durchzuführen.

Zuerst laden wir hierfür einen Datensatz mit dem Modul Pandas und bekommen einen kurzen Überblick über den Aufbau der Tabelle mit der `.head()` und der `.shape` Methode, die auf den Datensatz angewandt werden. Methoden sind Befehle, die speziell für einen Datentyp sind und sie zeichnen sich dadurch aus, dass sie mit einem Punkt an die Variable angehangt werden, auf die sie angewandt werden. In diesem Fall einen Pandas DataFrame. 

In [ ]:
import pandas as pd

penguins = pd.read_csv(
    "./penguins_classification.csv"
)  # Der Punkt ist ein 'relativer Pfad', der dazu führt, dass im selben Ordner nach den Daten gesucht wird, wie das Skript ist.
print(penguins.head())
print(penguins.shape)

Die `.head()` Methode zeigt uns die ersten Teile des Dataframes und dass die Tabelle drei Spalten enthält drei Spalten mit den Namen "Culmen Length (mm)", "Culmen Depth (mm)", "Species".

Die `.shape` Methode zeigt uns die Dimensionen des Dataframes an. Der Dataframe hat (Zeilen, Spalten). Also 342 Zeilen und 3 Spalten.

Weitere Einblicke können Methoden wie `.describe` geben. Die einige Deskriptive Statistiken ausgeben.

In [ ]:
penguins.describe()

Das wichtigste, was wir aus der `.describe()` Methode lernen können ist, ob die numerischen Variablen normalverteilt sind. Wenn die Zeile 50% (also der Median) und der Mittelwert ungefähr gleich sind sind die Daten ungefähr symmetrisch. Außerdem bekommen wir durch die Standardabweichung und die anderen Quartile (25%, 75%) wie die Daten streuen.

Die meisten der Befehle (siehe Spaltennamen des Outputs der `.describe` Methode) können auch einzeln auf Spalten angewandt werden, z.B.: 

In [ ]:
penguins["Culmen Length (mm)"].count()  # Anzahl der Einträge

In [ ]:
penguins["Culmen Depth (mm)"].max()  # höchster Wert

Über die Species Spalte bekommen wir mit `.describe()`keine Auskunft, da es sich um eine Kategoriale Variable handelt, die auch nicht z.B. <a href="https://de.wikipedia.org/wiki/Dummy-Variable">Dummy Kodiert</a> ist und auch dann wäre die Menge an Informationen, die gewonnen werden kann begrenzt. Alternativ können wir etwas mit `value_counts()` herausfinden.

In [ ]:
penguins["Species"].value_counts()

Ähnliches wie durch die `.describe()` Methode, können wir durch Plots lernen. der jointplot erlaubt uns einen Scatterplot von zwei Variablen zu erstellen, der die Beziehung zwischen zwei Variablen beleuchtet und Histogramme, die die Verteilung der einzelnen Variablen beleuchtet.

Hierzu muss eventuell erst das Modul Seaborn installiert werden. Das kann durch das entfernen des "#" und das Ausführen, der folgenden Zeile erreicht werden.

In [ ]:
# Falls seaborn nicht installiert ist vor der letzten Zeile dieser Zelle das "#" entfernen und einmal ausführen,
# danach "#" wieder hinzufügen.

#%conda install -c conda-forge seaborn

In [ ]:
?sns.jointplot

In [ ]:
import seaborn as sns

sns.jointplot(data=penguins, x="Culmen Length (mm)", y="Culmen Depth (mm)");

Die Verteilung im Scatterplot wirkt erstmal als ob kein spezieller Zusammenhang zwischen Schnabellänge und Tiefe besteht. Einen noch genaueren Überblick können wir bekommen, wenn wir den Scatterplot in der Mitte abhängig von den verschiedenen Species einfärben mit dem hue Argument.

In [ ]:
sns.jointplot(
    data=penguins, x="Culmen Length (mm)", y="Culmen Depth (mm)", hue="Species"
);

Wenn wir uns diese Verteilungen ansehen, könnte der Eindruck entstehen, dass (1) die Adelie, bzw. Chinstrap Pinguine eine andere Schnabelteife als Gentoo Pinguine haben, Gento bzw. Cinstrap Pinguine sich in der Schnabellänge Adelia unterscheiden; (2) dass die Schnabellängen und -tiefen innerhalb der Spezies miteinander korrelieren.

# Korrelation

Zuerst beschäftigen wir uns mit der Korrelation zwischen Schnabellänge und -tiefe in der Adelie. Hierfür müssen wir alle Adelie Pinguine herausfiltern und mit der `scipy.stats.pearsonr` Funktion des scipy Moduls die Korrelation errechnen.

In [ ]:
# Reminder Filtern, Schritt für Schritt

# 1. Welche Zeilen enthalten Adelie Pinguine

enthältAdelie = penguins["Species"] == "Adelie"
print(
    enthältAdelie
)  # Pandas Series mit True, falls in der Zeile ein Adelie Pinguin ist, False wenn nicht

In [ ]:
Adelie = penguins[enthältAdelie]
print(Adelie.head())

Q: Wie ist der Mittelwert und der Median der Culmen Length (mm)/Depth (mm) nur in der Species Adelie?

Q: Kannst du einen jointplot nur für die neue Adelie-Tabelle erstellen?

Weiter mit der Korrelation. Die `pearsonr` Funktion gibt einen r-Wert und einen p-Wert aus, die wir den Variablen `r` und `p_value` zuordnen und anschließend mit einem sogenannte f-string (f für formated) ausgeben. Der f-string gekennzeichnet durch ein f vor den "" erlaubt uns Variablen in den String einzufügen und zu formatieren, z.B. gibt :.2f an, dass 2 Nachkommastellen ausgegeben werden sollen.

In [ ]:
#%conda install -c conda-forge scipy

from scipy.stats import pearsonr

x = Adelie["Culmen Length (mm)"]
y = Adelie["Culmen Depth (mm)"]

r, p_value = pearsonr(x, y)

print(
    f"Die Korrelation zwischen Culmen Length (mm) und Culmen Depth (mm) ist {r:.2f} mit einem P-Wert von {p_value:.3f}."
)

Q: Wiederhole die Korrelation für die anderen beiden Pinguinspezies.

# T-Tests

T-Tests ermitteln, ob sich der Mittelwert von zwei Stichproben signifikant unterscheiden. Es gibt gepaarte und ungepaart, bzw. mit und ohne Messwiederholung. Die Annahme ist wichtig, da bei einer Messwiederholung ein Teil der Varianz durch das Individuum selbst, was dasselbe bleibt, erklärt werden kann. Im nächsten Teil werden wir untersuchen, ob sich die Schnabellänge von Adelie und Gentoo Pinguinen unterscheiden.

In [ ]:
from scipy.stats import ttest_ind

x = Adelie["Culmen Length (mm)"]

enthältGentoo = penguins["Species"] == "Gentoo"
Gentoo = penguins[enthältGentoo]
y = Gentoo["Culmen Length (mm)"]

t, p_value = ttest_ind(x, y, equal_var=False)

print(
    f"Der T-Test zwischen Adelie und Gentoo Schnabellänge ergibt einen t-Wert von {t:.2f} mit einem Signifikanzniveau von {p_value:.3f}"
)

Etwas ausführlichere Teststatistiken sind mit den Modul Pingouin möglich.

In [ ]:
#%conda install -c conda-forge pingouin

import pingouin as pg

pg.ttest(x, y, paired=False)

Q: Wiederhole die T-Tests und für alle weiteren möglichen Kombinationen

# Anmerkung

Normalerweise würde man es vermeiden mehrere T-Tests zu rechnen und stattdessen einen anderen Test rechnen, der den Gesamtendatensatz umspannt. Das ist vor allem deswegen der Fall, weil durch mehrere Tests der Alphafehler-Kummuliert, d.h. es wird wahrscheinlicher ein falsch-positives Ergebnis zu bekommen durch puren Zufall. Normalerweise gehen wir davon aus, dass ein p-Wert < 0.05 signifikant ist, d.h. 5 von 100 Tests sind falsch-positiv. Die einfachste Anpassung wäre einfach die Signifikanzgrenze (0.05) durch die Anzahl der Tests (die du oben durchgeführt hast) zu rechnen. 

Führe diese Anpassung der Signifikanzgrenze durch.

Q: Was bedeutet dies für die oben gerechneten T-Tests?

Q: Es gibt von Pingouin auch einen Korrelationsbefehl. Kannst du herausfinden (googlen) wie dieser anzuwenden ist und die Aufgabe von oben wiederholen?